In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os as os
import tensorflow as ts
import matplotlib.pyplot as plt

## Create Model

### Neural Network

- Several fully-connected layers

#### Output

##### Win-based

- 7 output nodes, predict number of wins (0-6)
- Apply softmax function at end to get probability

##### Rank-based

- Find the optimal ranking of teams for each year
- 11 output nodes, predict which rank (0-10) the team should be placed in

## Validate Results

- Want to pick several Cinderella teams for bottom 4-6 predictions
